<a href="https://colab.research.google.com/github/RoaaAlaa1/BeautyAdv/blob/master/beauty_advisor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving 360_F_276114144_xyAwCmx6IraojhvC2l9cNQEYAx6d28Nq.jpg to 360_F_276114144_xyAwCmx6IraojhvC2l9cNQEYAx6d28Nq (1).jpg


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image
import json

# Initialize face mesh
mp_face_mesh = mp.solutions.face_mesh

def extract_features(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            return {"error": "Could not read image"}

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
            result = face_mesh.process(image_rgb)
            if not result.multi_face_landmarks:
                return {"error": "No face detected"}

            landmarks = result.multi_face_landmarks[0].landmark

            # -------------------- FACE SHAPE --------------------
            jaw = [landmarks[i] for i in range(0, 17)]
            jaw_width = np.linalg.norm(np.array([jaw[0].x, jaw[0].y]) - np.array([jaw[-1].x, jaw[-1].y]))
            face_height = np.linalg.norm(np.array([landmarks[10].x, landmarks[10].y]) - np.array([landmarks[152].x, landmarks[152].y]))
            ratio = jaw_width / face_height
            if ratio < 0.85:
                face_shape = "Oval"
            elif 0.85 <= ratio < 1.05:
                face_shape = "Round"
            else:
                face_shape = "Square"

            # -------------------- SKIN TONE --------------------
            points = [(int(l.x * w), int(l.y * h)) for l in landmarks]
            cheeks = [points[234], points[454], points[93], points[323]]
            sample_pixels = []
            for x, y in cheeks:
                if 0 <= x < w and 0 <= y < h:
                    sample_pixels.append(image[y, x])
            if sample_pixels:
                avg_skin = np.mean(sample_pixels, axis=0).astype(int).tolist()
            else:
                avg_skin = None

            # -------------------- EYE COLOR --------------------
            left_eye_indices = [33, 133, 160, 159]
            right_eye_indices = [362, 263, 387, 386]
            eye_pixels = []

            for idxs in [left_eye_indices, right_eye_indices]:
                eye_pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in idxs]
                x_coords = [x for x, y in eye_pts]
                y_coords = [y for x, y in eye_pts]
                if x_coords and y_coords:
                    x1, x2 = min(x_coords), max(x_coords)
                    y1, y2 = min(y_coords), max(y_coords)
                    eye_crop = image[y1:y2, x1:x2]
                    if eye_crop.size > 0:
                        eye_pixels.append(np.mean(eye_crop.reshape(-1, 3), axis=0))

            if eye_pixels:
                eye_color = np.mean(eye_pixels, axis=0).astype(int).tolist()
            else:
                eye_color = None

            # -------------------- HAIR COLOR & TEXTURE --------------------
            hair_region = image[0:int(h * 0.15), int(w * 0.3):int(w * 0.7)]
            if hair_region.size > 0:
                hair_pixels = hair_region.reshape(-1, 3)
                avg_hair_color = np.mean(hair_pixels, axis=0).astype(int).tolist()

                # Estimate texture by sharpness
                gray_hair = cv2.cvtColor(hair_region, cv2.COLOR_BGR2GRAY)
                laplacian_var = cv2.Laplacian(gray_hair, cv2.CV_64F).var()
                if laplacian_var > 150:
                    hair_texture = "Curly or Coarse"
                elif laplacian_var > 50:
                    hair_texture = "Wavy or Slightly Curly"
                else:
                    hair_texture = "Straight or Smooth"
            else:
                avg_hair_color = None
                hair_texture = None

            return {
                "face_shape": face_shape,
                "skin_tone_rgb": avg_skin,
                "eye_color_rgb": eye_color,
                "hair_color_rgb": avg_hair_color,
                "hair_texture": hair_texture
            }

    except Exception as e:
        return {"error": f"Processing error: {str(e)}"}

In [3]:
image_path = list(uploaded.keys())[0]
features = extract_features(image_path)
print(json.dumps(features, indent=2))


{
  "face_shape": "Oval",
  "skin_tone_rgb": [
    132,
    154,
    193
  ],
  "eye_color_rgb": [
    80,
    94,
    123
  ],
  "hair_color_rgb": [
    155,
    161,
    170
  ],
  "hair_texture": "Curly or Coarse"
}


In [4]:
import json

data = {
    "face_shape": "Oval",
    "skin_tone_rgb": [132, 154, 193],
    "eye_color_rgb": [80, 94, 123],
    "hair_color_rgb": [155, 161, 170],
    "hair_texture": "Curly or Coarse"
}

# Save to file
with open("face_attributes.json", "w") as f:
    json.dump(data, f, indent=4)

print("Saved to face_attributes.json")


Saved to face_attributes.json


In [5]:
with open("face_attributes.json") as f:
    user_data = json.load(f)


In [6]:
def generate_recommendations(data):
    recommendations = {}

    # --- Makeup Style ---
    skin_rgb = data["skin_tone_rgb"]
    face_shape = data["face_shape"]
    eye_rgb = data["eye_color_rgb"]

    if skin_rgb[0] < 120:
        recommendations["makeup"] = "Use warm foundation shades, peach or coral blush, and nude or copper eyeshadow."
    else:
        recommendations["makeup"] = "Go with neutral or cool-toned foundation, rose blush, and light pink or grey eyeshadow."

    if eye_rgb[2] > 100:  # bluish
        recommendations["makeup"] += " Highlight blue eyes with brown or bronze tones."
    elif eye_rgb[1] > 100:  # green/hazel
        recommendations["makeup"] += " Use purple or warm brown eyeshadows for contrast."
    else:
        recommendations["makeup"] += " Try gold, olive, or navy eyeshadow to make brown eyes pop."

    # --- Hair Style ---
    hair_texture = data["hair_texture"]

    if face_shape == "Round":
        recommendations["hair_style"] = "Long layers or volume at the crown to elongate the face."
    elif face_shape == "Oval":
        recommendations["hair_style"] = "Most styles work well — try waves, layered bobs, or curtain bangs."
    elif face_shape == "Square":
        recommendations["hair_style"] = "Soft layers and waves to soften jawline."
    elif face_shape == "Heart":
        recommendations["hair_style"] = "Chin-length bobs or side-swept bangs work best."
    else:
        recommendations["hair_style"] = "Try side parts and gentle waves for balance."

    if "Curly" in hair_texture:
        recommendations["hair_style"] += " Embrace your curls with curl-defining products or try a curly shag cut."
    elif "Straight" in hair_texture:
        recommendations["hair_style"] += " Consider long layers or curtain bangs for volume."

    return recommendations


In [7]:
recommendations = generate_recommendations(data)
print(json.dumps(recommendations, indent=4))


{
    "makeup": "Go with neutral or cool-toned foundation, rose blush, and light pink or grey eyeshadow. Highlight blue eyes with brown or bronze tones.",
    "hair_style": "Most styles work well \u2014 try waves, layered bobs, or curtain bangs. Embrace your curls with curl-defining products or try a curly shag cut."
}
